In [2]:
import sqlparse

In [1]:
pip install sqlparse

  Using cached sqlparse-0.5.0-py3-none-any.whl.metadata (3.9 kB)
Using cached sqlparse-0.5.0-py3-none-any.whl (43 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
with open('../../../../Desktop/docu_str.sql', 'r') as file:
    content = file.read()
    def get_table_name(tokens):
        for token in reversed(tokens):
            if token.ttype is None:
                return token.value
        return " "

    parse = sqlparse.parse(content)
    for stmt in parse:
        # Get all the tokens except whitespaces
        tokens = [t for t in sqlparse.sql.TokenList(stmt.tokens) if t.ttype != sqlparse.tokens.Whitespace]
        is_create_stmt = False
        for i, token in enumerate(tokens):
            # Is it a create statements ?
            if token.match(sqlparse.tokens.DDL, 'CREATE'):
                is_create_stmt = True
                continue
            
            # If it was a create statement and the current token starts with "("
            if is_create_stmt and token.value.startswith("("):
                # Get the table name by looking at the tokens in reverse order till you find
                # a token with None type
                print (f"table: {get_table_name(tokens[:i])}")

                # Now parse the columns
                txt = token.value
                columns = txt[1:txt.rfind(")")].replace("\n","").split(",")
                for column in columns:
                    c = ' '.join(column.split()).split()
                    c_name = c[0].replace('\"',"")
                    if c_name == ['PRIMARY', 'KEY']:
                        c_type = 'KEY'
                    else:
                        c_type = c[1]  # For condensed type information 
                        # OR s
                    print (f"column: {c_name}")
                    print (f"data type: {c_type}")
                print ("---"*20)
                break

table: `aggregate_completed`
column: `col_name`
data type: varchar(10)
------------------------------------------------------------
table: `docu_acte_assure_stat_data`
column: `id_data`
data type: int(10)
column: `id_owner`
data type: int(10)
column: `sys_creation_date`
data type: datetime
column: `sys_last_mod_date`
data type: datetime
column: `sys_last_mod_user`
data type: int(10)
column: `ddn`
data type: date
column: `dep`
data type: varchar(255)
column: `id_assure`
data type: int(11)
column: `id_acte`
data type: int(11)
column: `date_acte`
data type: date
column: `type_test`
data type: int(10)
column: `num_ref`
data type: varchar(10)
column: `caisse`
data type: varchar(9)
column: `id_bci`
data type: varchar(14)
column: `insee`
data type: varchar(6)
column: `med_traitant`
data type: int(1)
column: `med_gyneco`
data type: int(1)
column: `npai`
data type: int(1)
column: `date_import`
data type: date
column: `invitrel_moins_1an`
data type: int(1)
column: `depistage_moins_3_5ans`
data t

IndexError: list index out of range

In [12]:
with open('../../../../Desktop/docu_str.sql', 'r') as file:
    content = file.read()
    parsed = sqlparse.parse(content)

In [13]:
for something in parsed:
    if something.get_type() == 'CREATE':
        print(something.tokens)

[<Newline ' ' at 0x14E409216C0>, <DDL 'CREATE' at 0x14E409218A0>, <Whitespace ' ' at 0x14E40921900>, <Keyword 'TABLE' at 0x14E40921960>, <Whitespace ' ' at 0x14E40921C00>, <Identifier '`aggre...' at 0x14E408F2250>, <Whitespace ' ' at 0x14E40921E40>, <Parenthesis '( `c...' at 0x14E408F2050>, <Whitespace ' ' at 0x14E409233A0>, <Keyword 'ENGINE' at 0x14E40923640>, <Comparison '=' at 0x14E40923820>, <Identifier 'InnoDB' at 0x14E408F22D0>, <Whitespace ' ' at 0x14E409238E0>, <Keyword 'DEFAULT' at 0x14E40923940>, <Whitespace ' ' at 0x14E409239A0>, <Keyword 'CHARSET' at 0x14E40923C40>, <Comparison '=' at 0x14E40923E20>, <Identifier 'latin1' at 0x14E408F21D0>, <Whitespace ' ' at 0x14E40923EE0>, <Keyword 'COLLATE' at 0x14E408D4040>, <Comparison '=' at 0x14E408D40A0>, <Identifier 'latin1...' at 0x14E408F2450>, <Punctuation ';' at 0x14E408D4160>]
[<Newline ' ' at 0x14E408D4C40>, <DDL 'CREATE' at 0x14E408D4CA0>, <Whitespace ' ' at 0x14E408D4D00>, <Keyword 'TABLE' at 0x14E408D4D60>, <Whitespace ' ' 

In [8]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False
            for token in tokens:
                if token.ttype is None and token.value.upper() == 'TABLE':
                    table_name = token.get_next_sibling().get_real_name()
                    table_info['table_name'] = table_name.strip('`')
                
                if token.ttype is None and token.value == '(':
                    in_parenthesis = True
                elif token.ttype is None and token.value == ')':
                    in_parenthesis = False
                
                if in_parenthesis:
                    if token.ttype is None and token.value.upper() == 'PRIMARY':
                        in_primary_key = True
                    elif in_primary_key and isinstance(token, sqlparse.sql.Identifier):
                        primary_keys.append(token.get_real_name())
                    elif token.ttype is None and token.value.upper() == 'FOREIGN':
                        in_foreign_key = True
                    elif in_foreign_key and isinstance(token, sqlparse.sql.Identifier):
                        foreign_keys.append(token.get_real_name())
                    elif isinstance(token, sqlparse.sql.Identifier):
                        column_name = token.get_real_name()
                        column_type = token.tokens[-1].get_real_name()
                        columns.append((column_name, column_type))
                
                if token.ttype is None and token.value.upper() == 'ENGINE':
                    break
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        print(f"Table: {table['table_name']}")
        headers = ["Column Name", "Data Type"]
        print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
        if table['primary_keys']:
            print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
        if table['foreign_keys']:
            print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
        print("\n")
        
# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

KeyError: 'table_name'

In [9]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False
            for token in tokens:
                if token.ttype is None and token.value.upper() == 'TABLE':
                    table_name = token.get_real_name()
                    table_info['table_name'] = table_name.strip('`')
                
                if token.ttype is None and token.value == '(':
                    in_parenthesis = True
                elif token.ttype is None and token.value == ')':
                    in_parenthesis = False
                
                if in_parenthesis:
                    if token.ttype is None and token.value.upper() == 'PRIMARY':
                        in_primary_key = True
                    elif in_primary_key and isinstance(token, sqlparse.sql.Identifier):
                        primary_keys.append(token.get_real_name())
                    elif token.ttype is None and token.value.upper() == 'FOREIGN':
                        in_foreign_key = True
                    elif in_foreign_key and isinstance(token, sqlparse.sql.Identifier):
                        foreign_keys.append(token.get_real_name())
                    elif isinstance(token, sqlparse.sql.Identifier):
                        column_name = token.get_real_name()
                        column_type = token.tokens[-1].get_real_name()
                        columns.append((column_name, column_type))
                
                if token.ttype is None and token.value.upper() == 'ENGINE':
                    break
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")
        
# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)


In [25]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False
            for token in tokens:
                print(f"Token: {token}")
                if token.ttype is None and token.value.upper() == 'TABLE':
                    table_name = token.get_real_name()
                    table_info['table_name'] = table_name.strip('`')
                    print(f"Found table name: {table_info['table_name']}")
                
                if token.ttype is None and token.value == '(':
                    in_parenthesis = True
                elif token.ttype is None and token.value == ')':
                    in_parenthesis = False
                
                if in_parenthesis:
                    if token.ttype is None and token.value.upper() == 'PRIMARY':
                        in_primary_key = True
                    elif in_primary_key and isinstance(token, sqlparse.sql.Identifier):
                        primary_keys.append(token.get_real_name())
                    elif token.ttype is None and token.value.upper() == 'FOREIGN':
                        in_foreign_key = True
                    elif in_foreign_key and isinstance(token, sqlparse.sql.Identifier):
                        foreign_keys.append(token.get_real_name())
                    elif isinstance(token, sqlparse.sql.Identifier):
                        column_name = token.get_real_name()
                        column_type = token.tokens[-1].get_real_name()
                        columns.append((column_name, column_type))
                
                if token.ttype is None and token.value.upper() == 'ENGINE':
                    break
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            
            print(f"Final table info: {table_info}")
            
            if table_name:
                tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

Token: 

Token: CREATE
Token:  
Token: TABLE
Token:  
Token: `aggregate_completed`
Token:  
Token: (
  `col_name` varchar(10) DEFAULT NULL
)
Token:  
Token: ENGINE
Token: =
Token: InnoDB
Token:  
Token: DEFAULT
Token:  
Token: CHARSET
Token: =
Token: latin1
Token:  
Token: COLLATE
Token: =
Token: latin1_swedish_ci
Token: ;
Final table info: {'columns': [], 'primary_keys': [], 'foreign_keys': []}
Token: 

Token: CREATE
Token:  
Token: TABLE
Token:  
Token: `docu_acte_assure_stat_data`
Token:  
Token: (
  `id_data` int(10) unsigned NOT NULL AUTO_INCREMENT,
  `id_owner` int(10) unsigned DEFAULT NULL,
  `sys_creation_date` datetime DEFAULT NULL,
  `sys_last_mod_date` datetime DEFAULT NULL,
  `sys_last_mod_user` int(10) unsigned DEFAULT NULL,
  `ddn` date DEFAULT NULL,
  `dep` varchar(255) DEFAULT NULL,
  `id_assure` int(11) DEFAULT NULL,
  `id_acte` int(11) DEFAULT NULL,
  `date_acte` date DEFAULT NULL,
  `type_test` int(10) unsigned DEFAULT NULL,
  `num_ref` varchar(10) DEFAULT NULL,
  `c

In [16]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_columns = False
            in_primary_key = False
            in_foreign_key = False

            for token in tokens:
                if token.is_keyword and token.value.upper() == 'TABLE':
                    table_name = token.get_real_name()
                    table_info['table_name'] = table_name.strip('`')
                
                if token.is_group:
                    for inner_token in token.flatten():
                        if inner_token.is_group:
                            if inner_token.value.upper() == 'PRIMARY KEY':
                                in_primary_key = True
                                in_columns = False
                                continue
                            elif inner_token.value.upper().startswith('FOREIGN KEY'):
                                in_foreign_key = True
                                in_columns = False
                                continue
                        
                        if in_columns:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                column_name = inner_token.get_real_name()
                                column_type = inner_token.tokens[-1].get_real_name()
                                columns.append((column_name, column_type))
                        elif in_primary_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                primary_keys.append(inner_token.get_real_name())
                        elif in_foreign_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                foreign_keys.append(inner_token.get_real_name())
                        
                        if inner_token.is_group and inner_token.value == '(':
                            in_columns = True
                        elif inner_token.is_group and inner_token.value == ')':
                            in_columns = False
                            in_primary_key = False
                            in_foreign_key = False
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            
            if table_name:
                tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)


AttributeError: 'Token' object has no attribute 'get_real_name'

In [17]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    # Parse the SQL statements
    parsed = sqlparse.parse(sql)
    
    tables = []
    
    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.flatten()
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []
            
            in_columns = False
            in_primary_key = False
            in_foreign_key = False

            for token in tokens:
                # Skip whitespace tokens
                if token.ttype is sqlparse.tokens.Whitespace:
                    continue

                if token.is_keyword and token.value.upper() == 'TABLE':
                    table_name = token.get_real_name()
                    table_info['table_name'] = table_name.strip('`')
                
                if token.is_group:
                    for inner_token in token.flatten():
                        if inner_token.is_group:
                            if inner_token.value.upper() == 'PRIMARY KEY':
                                in_primary_key = True
                                in_columns = False
                                continue
                            elif inner_token.value.upper().startswith('FOREIGN KEY'):
                                in_foreign_key = True
                                in_columns = False
                                continue
                        
                        if in_columns:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                column_name = inner_token.get_real_name()
                                column_type = inner_token.tokens[-1].get_real_name()
                                columns.append((column_name, column_type))
                        elif in_primary_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                primary_keys.append(inner_token.get_real_name())
                        elif in_foreign_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                foreign_keys.append(inner_token.get_real_name())
                        
                        if inner_token.is_group and inner_token.value == '(':
                            in_columns = True
                        elif inner_token.is_group and inner_token.value == ')':
                            in_columns = False
                            in_primary_key = False
                            in_foreign_key = False
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            
            if table_name:
                tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    # Read the SQL dump file
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    # Extract table information
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

AttributeError: 'Token' object has no attribute 'get_real_name'

In [18]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.flatten()
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []
            
            in_columns = False
            in_primary_key = False
            in_foreign_key = False

            for token in tokens:
                # Skip whitespace tokens
                if token.ttype is sqlparse.tokens.Whitespace:
                    continue

                if token.is_keyword and token.value.upper() == 'TABLE':
                    # Extract table name
                    table_name = tokens[tokens.index(token) + 2].value.strip('`')
                    table_info['table_name'] = table_name
                
                if token.is_group:
                    for inner_token in token.flatten():
                        if inner_token.is_group:
                            if inner_token.value.upper() == 'PRIMARY KEY':
                                in_primary_key = True
                                in_columns = False
                                continue
                            elif inner_token.value.upper().startswith('FOREIGN KEY'):
                                in_foreign_key = True
                                in_columns = False
                                continue
                        
                        if in_columns:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                column_name = inner_token.value.strip('`')
                                column_type = inner_token.tokens[-1].value
                                columns.append((column_name, column_type))
                        elif in_primary_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                primary_keys.append(inner_token.value.strip('`'))
                        elif in_foreign_key:
                            if isinstance(inner_token, sqlparse.sql.Identifier):
                                foreign_keys.append(inner_token.value.strip('`'))
                        
                        if inner_token.is_group and inner_token.value == '(':
                            in_columns = True
                        elif inner_token.is_group and inner_token.value == ')':
                            in_columns = False
                            in_primary_key = False
                            in_foreign_key = False
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            
            if table_name:
                tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

AttributeError: 'generator' object has no attribute 'index'

In [21]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False
            for token in tokens:
                if token.ttype is sqlparse.tokens.Whitespace:
                    continue
                
                if token.is_keyword and token.value.upper() == 'TABLE':
                    table_name = tokens[tokens.index(token) + 2].value.strip('`')
                    table_info['table_name'] = table_name
                
                if isinstance(token, sqlparse.sql.Parenthesis):
                    in_parenthesis = True
                    continue
                
                if in_parenthesis:
                    if isinstance(token, sqlparse.sql.Identifier):
                        if in_primary_key:
                            primary_keys.append(token.get_real_name().strip('`'))
                        elif in_foreign_key:
                            foreign_keys.append(token.get_real_name().strip('`'))
                        else:
                            column_name = token.get_real_name().strip('`')
                            column_type = token.tokens[-1].value
                            columns.append((column_name, column_type))
                    elif token.value.upper() == 'PRIMARY':
                        in_primary_key = True
                    elif token.value.upper() == 'FOREIGN':
                        in_foreign_key = True
                    elif token.value == ')':
                        in_parenthesis = False
                        in_primary_key = False
                        in_foreign_key = False
            
            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys
            
            if table_name:
                tables.append(table_info)
    
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

Table: aggregate_completed
+-------------------+-------------------+
|    Column Name    |     Data Type     |
+-------------------+-------------------+
|      InnoDB       |      InnoDB       |
|      latin1       |      latin1       |
| latin1_swedish_ci | latin1_swedish_ci |
+-------------------+-------------------+


Table: docu_acte_assure_stat_data
+--------------------+--------------------+
|    Column Name     |     Data Type      |
+--------------------+--------------------+
|       InnoDB       |       InnoDB       |
|      utf8mb3       |      utf8mb3       |
| utf8mb3_general_ci | utf8mb3_general_ci |
+--------------------+--------------------+


Table: docu_acte_assure_stat_data_group
+--------------------+--------------------+
|    Column Name     |     Data Type      |
+--------------------+--------------------+
|       InnoDB       |       InnoDB       |
|      utf8mb3       |      utf8mb3       |
| utf8mb3_general_ci | utf8mb3_general_ci |
+--------------------+-------

In [98]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []
    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False

            for token in tokens:
                if str(token).startswith('`'):
                    table_name = str(token).strip('`')
                    print(table_name)
                elif str(token).startswith('('):
                    #print(token)
                    liste_infos = str(token).strip('()').split('\n')
                    for info in liste_infos:
                        if info == '':
                            continue
                        else:
                            infos = info.strip().split(' ')
                            print(infos[0])
                            if infos[0].startswith('`'):
                                column_name = infos[0].strip('`')
                                columns_type = infos[1]
                                columns.append((column_name, columns_type))
                            elif infos[0] == 'PRIMARY':
                                print(info)
                                #print("lol")
                                #print(infos[0], infos[2].strip('(').strip('`').strip(',').strip(')').strip('`'))
                            elif infos[0] in ['KEY', 'UNIQUE']:
                                print(infos)
                            else:
                                print("hello")
    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        if 'table_name' in table:
            print(f"Table: {table['table_name']}")
            headers = ["Column Name", "Data Type"]
            print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
            if table['primary_keys']:
                print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
            if table['foreign_keys']:
                print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
            print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)

aggregate_completed
`col_name`
docu_acte_assure_stat_data
`id_data`
`id_owner`
`sys_creation_date`
`sys_last_mod_date`
`sys_last_mod_user`
`ddn`
`dep`
`id_assure`
`id_acte`
`date_acte`
`type_test`
`num_ref`
`caisse`
`id_bci`
`insee`
`med_traitant`
`med_gyneco`
`npai`
`date_import`
`invitrel_moins_1an`
`depistage_moins_3_5ans`
`npai_relance`
`depistage_primaire`
`date_deces`
`date_create`
`cmu`
`acte_insee`
PRIMARY
  PRIMARY KEY (`id_data`),
KEY
['KEY', '`date_acte`', '(`date_acte`)']
docu_acte_assure_stat_data_group
`id_data_group`
`id_data`
`id_group`
PRIMARY
  PRIMARY KEY (`id_data_group`),
UNIQUE
['UNIQUE', 'KEY', '`id_data`', '(`id_data`,`id_group`)']
docu_acte_assure_stat_data_group_mode
`id_data_group_mode`
`id_data_group`
`mode`
`date_valid`
PRIMARY
  PRIMARY KEY (`id_data_group_mode`),
UNIQUE
['UNIQUE', 'KEY', '`id_data_group`', '(`id_data_group`)']
docu_acte_assure_stat_deleted
`id_data`
`id_owner`
`sys_creation_date`
`sys_last_mod_date`
`sys_last_mod_user`
`ddn`
`dep`
`id_ass

In [76]:
import sqlparse
from tabulate import tabulate

def extract_table_info(sql):
    parsed = sqlparse.parse(sql)
    tables = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            table_info = {}
            tokens = statement.tokens
            
            table_name = None
            columns = []
            primary_keys = []
            foreign_keys = []

            in_parenthesis = False
            in_primary_key = False
            in_foreign_key = False

            for token in tokens:
                if str(token).startswith('CREATE TABLE'):
                    table_name = str(token).split()[2].strip('`')
                    table_info['table_name'] = table_name
                elif str(token).startswith('('):
                    # Extract columns and their types
                    column_info = str(token).strip('()').split(',')
                    for info in column_info:
                        if info.strip() == '':
                            continue
                        else:
                            info_parts = info.strip().split()
                            column_name = info_parts[0].strip('`')
                            column_type = ' '.join(info_parts[1:])
                            columns.append((column_name, column_type))

                # Extract primary keys
                if in_primary_key:
                    if str(token).startswith('PRIMARY KEY'):
                        pk_info = str(token).split()
                        for pk in pk_info[2:]:
                            if pk.startswith('(') or pk.endswith(')'):
                                pk = pk.strip('(),')
                            primary_keys.append(pk.strip('`'))

                # Extract foreign keys
                if in_foreign_key:
                    if str(token).startswith('FOREIGN KEY'):
                        fk_info = str(token).split()
                        for fk in fk_info[2:]:
                            if fk.startswith('(') or fk.endswith(')'):
                                fk = fk.strip('(),')
                            foreign_keys.append(fk.strip('`'))

                # Handle primary key and foreign key states
                if str(token).startswith('PRIMARY KEY'):
                    in_primary_key = True
                elif str(token).startswith('FOREIGN KEY'):
                    in_foreign_key = True
                elif str(token).startswith(')'):
                    in_primary_key = False
                    in_foreign_key = False

            table_info['columns'] = columns
            table_info['primary_keys'] = primary_keys
            table_info['foreign_keys'] = foreign_keys

            tables.append(table_info)

    return tables

def parse_sql_dump(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql = file.read()
    
    tables = extract_table_info(sql)
    return tables

def print_table_info(table_info):
    for table in table_info:
        print(f"Table: {table['table_name']}")
        headers = ["Column Name", "Data Type"]
        print(tabulate(table['columns'], headers=headers, tablefmt="pretty"))
        if table['primary_keys']:
            print(f"Primary Key(s): {', '.join(table['primary_keys'])}")
        if table['foreign_keys']:
            print(f"Foreign Key(s): {', '.join(table['foreign_keys'])}")
        print("\n")

# Example usage
sql_dump_path = '../../../../Desktop/docu_str.sql'
table_info = parse_sql_dump(sql_dump_path)
print_table_info(table_info)


KeyError: 'table_name'